In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

##### 准备数据

In [2]:
data   = pd.read_csv("dataset/HR.csv")

In [3]:
data   = data.join(pd.get_dummies(data.salary))
del    data["salary"]
data   = data.join(pd.get_dummies(data.part))
del    data["part"]

In [4]:
X_data = data[[c for c in data.columns if c != "left"]].values
Y_data = data[[c for c in data.columns if c == "left"]].values

##### 创建模型

In [5]:
from torch import nn
import torch.nn.functional as F

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(20,64)
        self.linear_2 = nn.Linear(64,64)
        self.linear_3 = nn.Linear(64,1)
    
    def forward(self,input):
        x = self.linear_1(input)
        x = F.relu(x)  
        x = self.linear_2(x)
        x = F.relu(x)
        x = self.linear_3(x)
        x = F.sigmoid(x) 
        return x

##### 训练模型

In [7]:
def get_model(): 
    model = Model() 
    opt   = torch.optim.Adam(model.parameters(),lr = 0.0001)  
    return model,opt

In [8]:
def accuracy(y_predict,y_true):
    y_predict = (y_predict>0.5).type(torch.int32)
    acc       = (y_predict == y_true).float().mean()
    return acc

In [9]:
model,opt         = get_model()
loss_fn           = nn.BCELoss()
batch_size        = 64
number_of_batches = len(data)//batch_size
epochs            = 100

##### 切分数据集并重新包装

In [10]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [11]:
train_x,test_x,train_y,test_y = train_test_split(X_data,Y_data)

In [12]:
train_x = torch.from_numpy(train_x).type(torch.float32)
train_y = torch.from_numpy(train_y).type(torch.float32)
test_x  = torch.from_numpy(test_x).type(torch.float32)
test_y  = torch.from_numpy(test_y).type(torch.float32)

In [13]:
train_ds = TensorDataset(train_x,train_y)
train_dl = DataLoader(train_ds,batch_size=batch_size,shuffle=True)
test_ds  = TensorDataset(test_x,test_y)
test_dl  = DataLoader(test_ds,batch_size=batch_size) 

##### 利用验证数据集对模型进行验证

In [14]:
for epoch in range(epochs):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x),train_y)
        epoch_loss = loss_fn(model(train_x),train_y).data
        
        epoch_test_accuracy = accuracy(model(test_x),test_y)
        epoch_test_loss = loss_fn(model(test_x),test_y).data
        
        print("epoch:","\t",epoch,"\t","loss: ","\t",round(epoch_loss.item(),3),"\t",
                             "accuracy: ","\t",round(epoch_accuracy.item(),3),"\t",
                             "test_loss: ","\t",round(epoch_test_loss.item(),3),"\t",
                             "test_accuracy: ","\t",round(epoch_test_accuracy.item(),3),"\t"
             )
        

D:\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch: 	 0 	 loss:  	 0.577 	 accuracy:  	 0.764 	 test_loss:  	 0.578 	 test_accuracy:  	 0.757 	
epoch: 	 1 	 loss:  	 0.569 	 accuracy:  	 0.764 	 test_loss:  	 0.571 	 test_accuracy:  	 0.757 	
epoch: 	 2 	 loss:  	 0.558 	 accuracy:  	 0.764 	 test_loss:  	 0.561 	 test_accuracy:  	 0.757 	
epoch: 	 3 	 loss:  	 0.556 	 accuracy:  	 0.764 	 test_loss:  	 0.559 	 test_accuracy:  	 0.757 	
epoch: 	 4 	 loss:  	 0.552 	 accuracy:  	 0.764 	 test_loss:  	 0.555 	 test_accuracy:  	 0.757 	
epoch: 	 5 	 loss:  	 0.548 	 accuracy:  	 0.764 	 test_loss:  	 0.551 	 test_accuracy:  	 0.757 	
epoch: 	 6 	 loss:  	 0.544 	 accuracy:  	 0.764 	 test_loss:  	 0.547 	 test_accuracy:  	 0.757 	
epoch: 	 7 	 loss:  	 0.537 	 accuracy:  	 0.764 	 test_loss:  	 0.541 	 test_accuracy:  	 0.757 	
epoch: 	 8 	 loss:  	 0.535 	 accuracy:  	 0.764 	 test_loss:  	 0.538 	 test_accuracy:  	 0.757 	
epoch: 	 9 	 loss:  	 0.527 	 accuracy:  	 0.764 	 test_loss:  	 0.53 	 test_accuracy:  	 0.757 	
epoch: 	 10

epoch: 	 84 	 loss:  	 0.274 	 accuracy:  	 0.888 	 test_loss:  	 0.283 	 test_accuracy:  	 0.887 	
epoch: 	 85 	 loss:  	 0.275 	 accuracy:  	 0.899 	 test_loss:  	 0.287 	 test_accuracy:  	 0.895 	
epoch: 	 86 	 loss:  	 0.266 	 accuracy:  	 0.899 	 test_loss:  	 0.276 	 test_accuracy:  	 0.897 	
epoch: 	 87 	 loss:  	 0.269 	 accuracy:  	 0.895 	 test_loss:  	 0.281 	 test_accuracy:  	 0.895 	
epoch: 	 88 	 loss:  	 0.264 	 accuracy:  	 0.9 	 test_loss:  	 0.275 	 test_accuracy:  	 0.897 	
epoch: 	 89 	 loss:  	 0.266 	 accuracy:  	 0.901 	 test_loss:  	 0.277 	 test_accuracy:  	 0.9 	
epoch: 	 90 	 loss:  	 0.264 	 accuracy:  	 0.902 	 test_loss:  	 0.275 	 test_accuracy:  	 0.9 	
epoch: 	 91 	 loss:  	 0.272 	 accuracy:  	 0.896 	 test_loss:  	 0.285 	 test_accuracy:  	 0.895 	
epoch: 	 92 	 loss:  	 0.261 	 accuracy:  	 0.904 	 test_loss:  	 0.272 	 test_accuracy:  	 0.899 	
epoch: 	 93 	 loss:  	 0.26 	 accuracy:  	 0.904 	 test_loss:  	 0.271 	 test_accuracy:  	 0.902 	
epoch: 